In [41]:
import pandas as pd
import polars as pl

In [42]:
fstec_vulnerabilities = pd.read_excel("Vull.xlsx", header=2)
fstec_vulnerabilities.head()

,Идентификатор,Наименование уязвимости,Описание уязвимости,Вендор ПО,Название ПО,Версия ПО,Тип ПО,Наименование ОС и тип аппаратной платформы,Класс уязвимости,Дата выявления,...,Возможные меры по устранению,Статус уязвимости,Наличие эксплойта,Информация об устранении,Ссылки на источники,Идентификаторы других систем описаний уязвимости,Прочая информация,Связь с инцидентами ИБ,Описание ошибки CWE,Тип ошибки CWE
0,BDU:2014-00001,Уязвимость микропрограммного обеспечения прогр...,Микропрограммное обеспечение модуля 140NOE7711...,Schneider Electric,Микропрограммное обеспечение программируемого ...,4.6 (Микропрограммное обеспечение программируе...,ПО программно-аппаратного средства АСУ ТП,Микропрограммное обеспечение программируемого ...,Уязвимость архитектуры,17.12.2011,...,Ограничение доступа к устройству по протоколу FTP,Подтверждена производителем,Существует,Информация об устранении отсутствует,http://ics-cert.us-cert.gov/alerts/ICS–ALERT–1...,CVE-2011-4859,Язык разработки ПО – С,0,Жесткое кодирование паролей,CWE-259
1,BDU:2014-00002,Уязвимость микропрограммного обеспечения маршр...,Скрипт «/scgi-bin/platform.cgi» микропрограммн...,D-Link Corp.,Микропрограммное обеспечение маршрутизатора D–...,1.02b11 (Микропрограммное обеспечение маршрути...,ПО сетевого программно-аппаратного средства,Микропрограммное обеспечение маршрутизатора D–...,Уязвимость кода,18.08.2013,...,Обновить программное обеспечение до версии 1.0...,Подтверждена производителем,Существует,Уязвимость устранена,http://www.exploit–db.com/papers/30061/,"CVE-2013-5946, OSVDB ID:100840, Bugtraq ID:641...",Язык разработки ПО – С,0,Проблемы обхода аутентификации,CWE-592
2,BDU:2014-00003,"Уязвимость браузера Opera, позволяющая злоумыш...","Браузер Opera содержит уязвимость, связанную с...",Opera Software ASA,Opera,до 11.65 включительно (Opera),Прикладное ПО информационных систем,"Windows (.), Mac OS (X)",Уязвимость архитектуры,14.06.2012,...,Обновление программного обеспечения до версии ...,Подтверждена производителем,Существует,Уязвимость устранена,http://www.opera.com/support/kb/view/1020/,"CVE-2012-3556, OSVDB ID:82952, Secunia Advisor...",Язык разработки ПО – С++,0,Недостаточная проверка вводимых данных,CWE-20
3,BDU:2014-00004,"Уязвимость браузера Firefox, позволяющая злоум...","Браузер Mozilla Firefox содержит уязвимость, к...",Mozilla Corp.,Firefox,12.0 (Firefox),Прикладное ПО информационных систем,"Linux (.), Windows (.)",Уязвимость архитектуры,06.05.2012,...,Обновление программного обеспечения до версии ...,Подтверждена производителем,Данные уточняются,Уязвимость устранена,http://www.mozilla.org/security/announce/2012/...,"CVE-2012-1944, OSVDB ID:82672, Secunia Advisor...","Язык разработки программного средства – С++, X...",0,Непринятие мер по защите структуры веб-страниц...,CWE-79
4,BDU:2014-00005,"Уязвимость гипервизора VMware Workstation, поз...",Злоумышленник может получить контроль над выпо...,VMware Inc.,VMWare Workstation,"6.5 (VMWare Workstation), от 7.0.0 до 7.1.4 вк...",ПО виртуализации/ПО виртуального программно-ап...,Linux (.),Уязвимость архитектуры,04.04.2011,...,Обновление программного обеспечения до версии ...,Подтверждена производителем,Данные уточняются,Уязвимость устранена,http://lists.vmware.com/pipermail/security-ann...,CVE-2011-1126,Язык разработки программного средства – С++,0,"Разрешения, привилегии и средства управления д...",CWE-264


In [43]:
fstec_vulnerabilities["date"] = pd.to_datetime(fstec_vulnerabilities["Дата выявления"], format="%d.%m.%Y")
fstec_vulnerabilities["cve_id"] = fstec_vulnerabilities["Идентификаторы других систем описаний уязвимости"].str.extractall(
    r"(CVE-\d{4}-\d{4})"
)
fstec_vulnerabilities = fstec_vulnerabilities[["Идентификатор", "Тип ошибки CWE", "Ссылки на источники", "cve_id", "date"]]
fstec_vulnerabilities.rename(
    columns={"Идентификатор": "fstec_id", "Тип ошибки CWE": "cwe_id", "Ссылки на источники": "reference"}, inplace=True
)
fstec_vulnerabilities["cve_id"] = fstec_vulnerabilities["cve_id"].astype("str")
fstec_vulnerabilities["cwe_id"] = fstec_vulnerabilities["cwe_id"].astype("str")
fstec_vulnerabilities["reference"] = fstec_vulnerabilities["reference"].astype("str")
fstec_vulnerabilities = pl.from_pandas(fstec_vulnerabilities, schema_overrides={"date": pl.Datetime, "reference": str})

TypeError: incompatible index of inserted column with frame index

In [ ]:
fstec_vulnerabilities = fstec_vulnerabilities.sort(by=["date"], descending=True)
fstec_vulnerabilities = fstec_vulnerabilities.drop_nulls(subset=["reference"])
fstec_vulnerabilities.describe()

[Utf8, Utf8, Utf8, Utf8, Datetime(time_unit='ns', time_zone=None)]

In [40]:
references = fstec_vulnerabilities.with_columns(
    pl.col("reference").str.replace(r"[\n]", r"\t").str.split(by="\t")
).explode(columns="reference").select("reference").to_series().to_list()

In [9]:
fstec_vulnerabilities["Ссылки на источники"].

,Идентификатор,Тип ошибки CWE,Дата выявления,Ссылки на источники,Идентификаторы других систем описаний уязвимости,cve_id
55100,BDU:2024-01671,CWE-400,2024-02-28,https://sec.cloudapps.cisco.com/security/cente...,CVE-2024-20321,CVE-2024-2032
55073,BDU:2024-01644,CWE-269,2024-02-27,https://www.teamviewer.com/en/trust-center/sec...,CVE-2024-0819,CVE-2024-0819
55033,BDU:2024-01604,CWE-78,2024-02-26,https://fontforge.org/en-US/downloads/\t\nhttp...,CVE-2024-25081,CVE-2024-2508
55038,BDU:2024-01609,CWE-276,2024-02-26,https://github.com/cyu/rack-cors/issues/274\nh...,CVE-2024-27456,CVE-2024-2745
55026,BDU:2024-01597,CWE-441,2024-02-26,https://www.moxa.com/en/support/product-suppor...,CVE-2024-0387,CVE-2024-0387
...,...,...,...,...,...,...
6438,BDU:2015-05954,NaN,NaT,http://lists.opensuse.org/opensuse-security-an...,"CVE-2014-6271, CVE-2014-7169, CVE-2014-7186, C...",CVE-2014-6271
6439,BDU:2015-05955,NaN,NaT,http://lists.opensuse.org/opensuse-security-an...,"CVE-2014-6271, CVE-2014-7169, CVE-2014-7186, C...",CVE-2014-6271
6440,BDU:2015-05956,NaN,NaT,http://lists.opensuse.org/opensuse-security-an...,"CVE-2014-6271, CVE-2014-7169, CVE-2014-7186, C...",CVE-2014-6271
6441,BDU:2015-05957,NaN,NaT,http://lists.opensuse.org/opensuse-security-an...,"CVE-2014-6271, CVE-2014-7169, CVE-2014-7186, C...",CVE-2014-6271


In [22]:
fstec_vulnerabilities[fstec_vulnerabilities["Ссылки на источники"].str.contains("github")]

ValueError: Cannot mask with non-boolean array containing NA / NaN values